### 檢查遺漏的題目

In [14]:
import pandas as pd

# 讀取三個 CSV 檔案
# df1 = pd.read_csv('../output_communitysearch.csv', header=None)
# df2 = pd.read_csv('../output_steinertree.csv', header=None)
df1 = pd.read_csv('/home/yuu0223/MindMap_Test/output_MindMap_Gemini15Flash.csv', header=0)
df2 = pd.read_csv('/home/yuu0223/MindMap_Test/output_Greedy_PR_KG.csv', header=0)
df3 = pd.read_csv('/home/yuu0223/MindMap_Test/output_Greedy_PR_subgraph.csv', header=0)

# # 產生題號範圍 30~129
selected_indices = set(range(31, 530))

# 檢查每個檔案中缺少的題號
# missing_in_df1 = expected_questions - set(df1[0])
# missing_in_df2 = expected_questions - set(df2[0])
missing_in_df1 = selected_indices - set(df1['Question_ID'])
missing_in_df2 = selected_indices - set(df2['Question_ID'])
missing_in_df3 = selected_indices - set(df3['Question_ID'])


# 顯示結果
# print("communitysearch 缺少的題號:", sorted(missing_in_df1))
# print("steinertree 缺少的題號:", sorted(missing_in_df2))
print("Original sort: \n")
print("MindMap 缺少的題號:", sorted(missing_in_df1))
print("Whole KG 缺少的題號:", sorted(missing_in_df2))
print("Subgraph 缺少的題號:", sorted(missing_in_df3))

Original sort: 

MindMap 缺少的題號: [51, 276]
Whole KG 缺少的題號: [51, 168, 277, 336]
Subgraph 缺少的題號: [51, 168, 277, 336]


### 合併檔案

In [16]:
import pandas as pd

# 讀取三個 CSV 檔案
# df1 = pd.read_csv('../output_communitysearch.csv', header=None)
# df1.columns = ["number", "question", "reference_ans", "output"]
# df2 = pd.read_csv('../output_steinertree.csv', header=None)
# df2.columns = ["number", "question", "reference_ans", "output"]


# 只保留 df2 和 df3 的 "number" 和 "output" 欄位，並重命名 output 為 output2 和 output3
# df1 = df1.sort_values(by='number')
# df2 = df2[['number', 'output']].rename(columns={'output': 'output2'}).sort_values(by='number')
# df3 = df3[['number', 'output']].rename(columns={'output': 'output3'}).sort_values(by='number')
df1 = df1.sort_values(by='Question_ID')
df2 = df2[['Question_ID', 'Answer']].rename(columns={'output': 'output2'}).sort_values(by='Question_ID')
df3 = df3[['Question_ID', 'Answer']].rename(columns={'output': 'output3'}).sort_values(by='Question_ID')

# 依照 "number" 進行合併，保留第一個檔案的所有欄位
# merged_df = df1.merge(df2, on='number', how='inner')
# merged_df = merged_df.merge(df3, on='number', how='inner')
merged_df = df1.merge(df2, on='Question_ID', how='inner')
merged_df = merged_df.merge(df3, on='Question_ID', how='inner')
merged_df.tail(5)

# 將合併的結果存成一個新的 CSV 檔案
merged_df.to_csv('./1_merge_file/merged_output_compare_PR500.csv', index=False)

### Original Evaluation Start

In [ ]:
# !pip install transformers

In [ ]:
# !pip install datasets

In [ ]:
# !pip install evaluate

In [ ]:
# !pip install openai

GPT-4 ranking. The output of MindMap only choose the summary result (Output1)

In [22]:
import openai
import csv
from time import sleep
import os

input_file = './1_merge_file/merged_output_compare_PR500.csv'
# input_file = '../output_SteinerTree_overlap_v2.csv' ###
output_file = './2_GPT_Ranking/output_ranking_PR500_round2.csv'

openai.api_key = os.getenv("openai_api_key")

def prompt_ranking(reference_text,output1_text,output2_text,output3_text): ###
  prompt_template = """
  Reference: {reference}

  output1: {output1}

  output2: {output2}

  output3: {output3}


  According to the facts of the disease and the drug and test recommendations in reference output, order the fact match of the output from highest to lowest.

  The output followed the format bellow：

  1. Output2
  2. Output3
  3. Output1
  """

  prompt = prompt_template.format(reference=reference_text, 
                                  output1=output1_text, 
                                  output2=output2_text,
                                  output3=output3_text) ###
  response = openai.ChatCompletion.create(
      model="gpt-4o",
      messages=[
          {"role": "system", "content": "You are a excellent."}, ### You are a skilled medical doctor reviewing the diagnoses made by other doctors for patients. Each output represents the diagnosis made by each doctor for the patient
          {"role": "user", "content": prompt}
      ]
  )

  generated_text = response.choices[0].message.content
  return generated_text

with open(input_file,'r',newline="") as f_input, open(output_file, 'a+', newline='') as f_output:
  reader = csv.reader(f_input)
  writer = csv.writer(f_output)

  header = next(reader)
  header.append("gpt4o_ranking")
  writer.writerow(header)

  for row in reader:
    reference_text = [row[2].strip("\n")]
    output1_text = [row[3].strip("\n")]
    output2_text = [row[4].strip("\n")]
    output3_text = [row[5].strip("\n")] ###

    flag = 0
    while flag == 0:
      try:
        str1 = prompt_ranking(reference_text,output1_text,output2_text,output3_text) ###
        flag = 1
      except:
        sleep(40)
        str1 = prompt_ranking(reference_text,output1_text,output2_text,output3_text) ###

    print(str1)
    print('\n')

    row.append(str1)
    writer.writerow(row)




Based on the comparison of the given


1. Output1
2. Output3
3. Output2


1. Output2  
   - Output2 is the highest match because it accurately addresses the core concern from the reference by suggesting radiographic imaging procedures (like CT scans or MRIs) to investigate potential intracranial abnormalities, which aligns directly with the reference's mention of running tests, including imaging, to explore the cause of headaches. It further suggests a detailed history and neurological examination, closely following the need for thorough investigation. 

2. Output3  
   - Output3 comes second as it mentions a CT scan of the sinuses to rule out sinusitis, which somewhat aligns with the mention in the reference about performing tests. Although it includes a sinus-related focus, it also discusses the need for a neurological examination to explore other headache causes, similar to the reference's intent, though it does not emphasize intracranial imaging as heavily as Output2.

3. Output1  

In [ ]:
import re
import csv

input_file = './2_GPT_Ranking/output_ranking_PR500_round2.csv' ###
output_file = './3_Ranking_Compute/output_ranking_compute_PR500_round2.csv' ###

with open(input_file,'r',newline="") as f_input, open(output_file, 'a+', newline='') as f_output:
  reader = csv.reader(f_input)
  writer = csv.writer(f_output)

  header = next(reader)
  header.extend(["output1_ranking","output2_ranking","output3_ranking"]) ###
  writer.writerow(header)

  for row in reader:
    if row[6] != 'gpt4o_ranking': ###
      output_text = row[6] ###
      print(output_text)

      
      matches = re.findall(r'^\d+\.\s*(Output\d+)(?::)?', output_text, re.MULTILINE)
      if matches == []:
        matches = re.findall(r'^\d+\.\s*\*\*(Output\d+)\*\*(?::)?', output_text, re.MULTILINE)
      if matches == []:
        matches = re.findall(r'^\d+\.\s*\*\*(Output\d+):\*\*', output_text, re.MULTILINE)


      rankings = {}
      for i, match in enumerate(matches):
        rank = i + 1
        output = match.replace(" ","")
        print(output)
        rankings[output] = rank

      output_dic = {}

      for output, rank in sorted(rankings.items(), key=lambda x: x[1]):
        print(f"{output}: {rank}")
        output_dic[output]=rank



      print(output_dic)
      print('\n')

      try:
        row.extend([output_dic['Output1'],output_dic['Output2'],output_dic['Output3']]) ###
        writer.writerow(row)

      except KeyError:
        ### 如果沒有回答，先算平手
        if output_dic == {}:
          row.extend([2,2,2]) ###
          writer.writerow(row)
        else:
          continue


Based on the comparison of the given
{}


1. Output1
2. Output3
3. Output2
Output1
Output3
Output2
Output1: 1
Output3: 2
Output2: 3
{'Output1': 1, 'Output3': 2, 'Output2': 3}


1. Output2  
   - Output2 is the highest match because it accurately addresses the core concern from the reference by suggesting radiographic imaging procedures (like CT scans or MRIs) to investigate potential intracranial abnormalities, which aligns directly with the reference's mention of running tests, including imaging, to explore the cause of headaches. It further suggests a detailed history and neurological examination, closely following the need for thorough investigation. 

2. Output3  
   - Output3 comes second as it mentions a CT scan of the sinuses to rule out sinusitis, which somewhat aligns with the mention in the reference about performing tests. Although it includes a sinus-related focus, it also discusses the need for a neurological examination to explore other headache causes, similar to the ref

In [ ]:
import pandas as pd

def calculate_column_average(file_path, column_name):
    df = pd.read_csv(file_path)
    average = df[column_name].mean()
    return average

column =["output1_ranking","output2_ranking","output3_ranking"] ###

file_path = './3_Ranking_Compute/output_ranking_compute_PR500.csv'
for column_name in column:
  average = calculate_column_average(file_path, column_name)
  print(f"The average of column {column_name} is: {round(average,3)}")

The average of column output1_ranking is: 1.625
The average of column output2_ranking is: 1.944
The average of column output3_ranking is: 2.468


In [24]:
import pandas as pd

def calculate_column_average(file_path, column_name):
    df = pd.read_csv(file_path)
    average = df[column_name].mean()
    return average

column =["output1_ranking","output2_ranking","output3_ranking"] ###

file_path = './3_Ranking_Compute/output_ranking_compute_PR500_round2.csv'
for column_name in column:
  average = calculate_column_average(file_path, column_name)
  print(f"The average of column {column_name} is: {round(average,3)}")

The average of column output1_ranking is: 1.694
The average of column output2_ranking is: 1.936
The average of column output3_ranking is: 2.426
